<a href="https://colab.research.google.com/github/cserock/colab-examples/blob/main/07_LangChain_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경변수 파일(.env) 설정하기
/content/drive/MyDrive/lg-dx/에 아래 내용으로 .env 파일을 작성한 후 업로드합니다.  

OPENAI_API_KEY=sk-xxxx  
LANGCHAIN_TRACING_V2="true"  
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"  
LANGCHAIN_API_KEY="lsv2_xxxx  
LANGCHAIN_PROJECT="lg-dx"


Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

python-dotenv 라이브러리 설치

In [ ]:
!pip install python-dotenv

환경변수 파일 로드 및 확인

In [ ]:
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/lg-dx/.env', override=True)

import os
openai_api_key = os.environ.get('OPENAI_API_KEY')
print("openai_api_key : " + openai_api_key)
langchain_api_key = os.environ.get('LANGCHAIN_API_KEY')
print("langchain_api_key : " + langchain_api_key)

LangChain 설치 및 업데이트

In [ ]:
# LangChain 설치 및 업데이트
!pip install -qU langchain-teddynote

설치된 LangChain 버전을 확인합니다.

In [ ]:
from importlib.metadata import version

print("[LangChain 관련 패키지 버전]")
for package_name in [
    "langchain",
    "langchain-core",
    "langchain-experimental",
    "langchain-community",
    "langchain-openai",
    "langchain-teddynote",
    "langchain-huggingface",
    "langchain-google-genai",
    "langchain-anthropic",
    "langchain-cohere",
    "langchain-chroma",
    "langchain-elasticsearch",
    "langchain-upstage",
    "langchain-cohere",
    "langchain-milvus",
    "langchain-text-splitters",
]:
    try:
        package_version = version(package_name)
        print(f"{package_name}: {package_version}")
    except ImportError:
        print(f"{package_name}: 설치되지 않음")

LangSmith 추적 설정  
.env 파일에 LangSmith API 키가 설정 되어 있어야 합니다.(LANGCHAIN_API_KEY)

In [ ]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("lg-dx-langchain")

아래와 같이 추적을 끌 수 있습니다.

In [ ]:
from langchain_teddynote import logging

# set_enable=False 로 지정하면 추적을 하지 않습니다.
logging.langsmith("lg-dx-langchain", set_enable=False)

## OpenAI API

In [ ]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o-mini",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
response = llm.invoke(question)
print(f"[답변]: {response}")

In [ ]:
# content 출력
print(response.content)

In [ ]:
# metadata 출력
print(response.response_metadata)

### LogProb 활성화
주어진 텍스트에 대한 모델의 토큰 확률의 로그 값 을 의미합니다. 토큰이란 문장을 구성하는 개별 단어나 문자 등의 요소를 의미하고, 확률은 모델이 그 토큰을 예측할 확률을 나타냅니다.

In [ ]:
# 객체 생성
llm_with_logprob = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o-mini",  # 모델명
).bind(logprobs=True)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
response = llm_with_logprob.invoke(question)

# 결과 출력
response.response_metadata


### 스트리밍 출력
스트리밍 옵션은 질의에 대한 답변을 실시간으로 받을 때 유용합니다.

In [ ]:
# 스트림 방식으로 질의
# answer 에 스트리밍 답변의 결과를 받습니다.
answer = llm.stream("대한민국의 아름다운 관광지 10곳과 주소를 알려주세요!")

# 스트리밍 방식으로 각 토큰을 출력합니다. (실시간 출력)
for token in answer:
    print(token.content, end="", flush=True)

### 멀티모달 모델(이미지 인식)
멀티모달은 여러 가지 형태의 정보(모달)를 통합하여 처리하는 기술이나 접근 방식을 의미합니다. 이는 다음과 같은 다양한 데이터 유형을 포함할 수 있습니다.  

- 텍스트: 문서, 책, 웹 페이지 등의 글자로 된 정보  
- 이미지: 사진, 그래픽, 그림 등 시각적 정보  
- 오디오: 음성, 음악, 소리 효과 등의 청각적 정보  
- 비디오: 동영상 클립, 실시간 스트리밍 등 시각적 및 청각적 정보의 결합

In [ ]:
from langchain_teddynote.models import MultiModal
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o",  # 모델명
)

# 멀티모달 객체 생성
multimodal_llm = MultiModal(llm)

# 샘플 이미지 주소(웹사이트로 부터 바로 인식)
IMAGE_URL = "https://t3.ftcdn.net/jpg/03/77/33/96/360_F_377339633_Rtv9I77sSmSNcev8bEcnVxTHrXB4nRJ5.jpg"

# 이미지 파일로 부터 질의
answer = multimodal_llm.stream(IMAGE_URL)

# 스트리밍 방식으로 각 토큰을 출력합니다. (실시간 출력)
stream_response(answer)

### System, User 프롬프트 수정

In [ ]:
system_prompt = """당신은 표(재무제표) 를 해석하는 금융 AI 어시스턴트 입니다.
당신의 임무는 주어진 테이블 형식의 재무제표를 바탕으로 흥미로운 사실을 정리하여 친절하게 답변하는 것입니다."""

user_prompt = """당신에게 주어진 표는 회사의 재무제표 입니다. 흥미로운 사실을 정리하여 답변하세요."""

# 멀티모달 객체 생성
multimodal_llm_with_prompt = MultiModal(
    llm, system_prompt=system_prompt, user_prompt=user_prompt
)

# 이미지의 경로 입력
IMAGE_PATH_FROM_FILE = "https://storage.googleapis.com/static.fastcampus.co.kr/prod/uploads/202212/080345-661/kwon-01.png"

# 이미지 파일로 부터 질의(스트림 방식)
answer = multimodal_llm_with_prompt.stream(IMAGE_PATH_FROM_FILE)

# 스트리밍 방식으로 각 토큰을 출력합니다. (실시간 출력)
stream_response(answer)